# Manual of root_segmentor_VIB  (NEW CAMERA)

NOTE: this manual is identical to 'tutorial.ipynb' but is adapted to the new camera setup (basically means that a new set of parameters is needed for creating training data, computing features, selecting roots etc.)

### Sep 0: imports

Import the required modules

In [1]:
import os
import sys
sys.path.append("..")
import root_segmentor_VIB as rs
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib tk


### Step 1: look at some sample data

The method `load_training_image` allows to read an image and the tracings for training as well. The `auto_transform` option allows to automatically transform the tracings coordinates into the coordinate system of the image.

In [2]:
im, names, vertices_s, vertices_e = rs.load_training_image(img_file = "../sample_newMachine/AG00IHWX_000000.jpg",
                                                        root_traces_file = "../sample_newMachine/AG00IHWX_000000 vertices.csv",
                                                        auto_transform=False)

#transform into row-column coordinates
vertices_s_RC = rs.flip_XY_RC(vertices_s)
vertices_e_RC = rs.flip_XY_RC(vertices_e)

To create training data from the loaded images, the tracings are first transformed into a root-mask with `create_root_mask` after which the function `create_root_buffer_background_image` transforms this into an image containing root-pixels (1), relevant background pixels for making a classification (2) and unclassified pixels (3). To do that, buffer zones are used around the images.

Lastly, the function `create_training_data` finally selects the left half of the image for training purposes. Can be skipped if entire image should be used for training.

In [3]:
# create root mask
root_mask = rs.create_root_mask(im, vertices_s_RC, vertices_e_RC, dilatation_radius= 2)
root_buffer_background = rs.create_root_buffer_background_image(root_mask, buffer_radius = 5, no_root_radius = 30)

# create training data labels
training_labels = rs.create_training_data(root_buffer_background)

The function `show_traces` allows to plot an image with the traincing on top (similar to imshow). Use `%matplotlib tk` for pop-up viewer

In [4]:
plt.subplot(1, 3, 1)
rs.show_traces(vertices_s, vertices_e, im)
plt.subplot(1, 3, 2)
rs.show_traces(vertices_s, vertices_e, root_buffer_background)
plt.subplot(1, 3, 3)
rs.show_traces(vertices_s, vertices_e, training_labels)

### Step 2: Compile a dataset for training

The tracings of multiple images are combined to learn a pixel-classifier. To achieve this goal, the following steps are taken:
* All images and tracings in `some_folder` are listed
* Per image, pixel-level features are computed (texture, gradient image etc.)
* The per image, the function `create_root_buffer_background_image` is used to comput the label of every pixel
* A fraction is points is sampled (reducing training dataset size and rebalancing it somewhat)
* The preveous steps are applied to all images in `some_folder` and combined in a features dataset `X` and labels dataset `Y`

The first step only computes labels and features per image and stores them as `npy` files.

In [5]:

# compute FEATURES and LABELS for each image in a given folder
files_list = ['../sample_newMachine/'+f for f in os.listdir('../sample_newMachine') if f[-3:] == "jpg"]
rs.imgs_to_XY_data(img_file_list = files_list,
                    root_traces_file_list = None,
                    auto_transform = False,
                    dilatation_radius = 2,
                    buffer_radius = 5,
                    no_root_radius = 30,
                    sigma_max = 10)

c:\GIT\Research\root_segmentor\tests\..\root_segmentor_VIB.py:58: UserWarning: loadtxt: input contained no data: "../sample_newMachine/DMSO000011 vertices.csv"
  vertices = np.loadtxt(root_traces_file,
c:\GIT\Research\root_segmentor\tests\..\root_segmentor_VIB.py:62: UserWarning: loadtxt: input contained no data: "../sample_newMachine/DMSO000011 vertices.csv"
  names = np.loadtxt(root_traces_file,


Problem processing ../sample_newMachine/DMSO000011.jpg


Traceback (most recent call last):
  File "c:\GIT\Research\root_segmentor\tests\..\root_segmentor_VIB.py", line 649, in imgs_to_XY_data
    training_labels = create_training_data(root_buffer_background)
  File "c:\GIT\Research\root_segmentor\tests\..\root_segmentor_VIB.py", line 247, in create_training_data
    low_idx = np.min(tmp)
  File "c:\Users\jverwaer\.conda\envs\napari-env\lib\site-packages\numpy\core\fromnumeric.py", line 2953, in min
    return _wrapreduction(a, np.minimum, 'min', axis, None, out,
  File "c:\Users\jverwaer\.conda\envs\napari-env\lib\site-packages\numpy\core\fromnumeric.py", line 88, in _wrapreduction
    return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
ValueError: zero-size array to reduction operation minimum which has no identity


The second step combines the generated files to create `X` and `Y`

In [6]:
# create training datasets
features_file_list = ['../sample_newMachine/'+f for f in os.listdir('../sample_newMachine') if f[-3:] == "npy" and "FEATURES" in f]
X, Y = rs.compile_training_dataset_from_precomputed_features(features_file_list, sample_fraction=(1.0, 1.0))

### Step 3: Train a model and save it

The compiled dataset is used to train a random forest classifier

In [7]:
# fit random forest classifier
clf = rs.RandomForestClassifier(n_estimators=100, n_jobs=-1,
                            max_depth=10, max_samples=0.05)
clf.fit(X, Y)
# dump the model to a file
rs.dump(clf, '../models/RF_AGO_000and002_NTrees-100.joblib')

['../models/RF_AGO_000and002_NTrees-100.joblib']

### Step 4: Load a saved model

Select a saved model and load it

In [8]:
clf = rs.load('../models/RF_AGO_000and002_NTrees-100.joblib')

### Step 5: Load new image to make predictions (and compare with tracings)

In [10]:
im = rs.io.imread("../sample_newMachine/test_cases/AG00IHWX_000000.jpg")
# compute features
features = rs.im2features(im, sigma_max = 10)
# predict
predicted_segmentation = rs.predict_segmentor(clf, features)
# clean detected roots
roots = rs.clean_predicted_roots(predicted_segmentation, small_objects_threshold=150, closing_diameter = 4)

Visualize the results

In [11]:
# draw detected roots
im_out = rs.draw_detected_roots(roots, im, root_thickness = 7, minimalBranchLength = 10)
# measure root properties and show as table
rs.measure_roots(roots, root_thickness = 7, minimalBranchLength = 10)

,X,Y,orientation,length,approx_check,adjusted_length,adjusted_length_with_sqrt2
0,1477.003303,183.042944,0.038545,256.001646,Bad approx,572.000000,600.409163
1,1360.178751,1115.299354,0.295203,150.066927,OK,150.066927,150.066927
2,1469.973205,1102.600214,0.203327,95.629663,OK,95.629663,95.629663
3,1513.391931,1119.517291,0.083014,78.477130,OK,78.477130,78.477130
4,1191.893568,1183.588055,0.153539,132.299225,OK,132.299225,132.299225
5,1020.550196,1209.668536,0.189335,170.854799,OK,170.854799,170.854799
6,1348.147399,1151.939306,0.208847,43.596617,OK,43.596617,43.596617
7,1081.308370,1254.643172,-0.203824,102.115058,OK,102.115058,102.115058
8,1256.787565,1220.134715,0.080209,27.697022,OK,27.697022,27.697022
9,733.681564,1313.622905,0.327590,181.758115,OK,181.758115,181.758115


### Step 6: Export the results to a file

The lenths, orientation, position etc. of the roots can be exported to a file

In [8]:
results_df = rs.measure_roots(roots)
results_df.to_excel("../sample_results/measurements.xlsx")

### Step 7: Automate classification per folder

Lists all files in `some_dir`, detects roots and saves the results in a xlsx file. All detected roots are saved for quality checking.

In [6]:
# list all test images in folder
features_file_list = ['../sample_newMachine/test_cases/'+f for f in os.listdir('../sample_newMachine/test_cases') if f[-3:] == "jpg"]

all_results = []

for fname in features_file_list:
    # read image
    im = rs.io.imread(fname)
    # compute features
    features = rs.im2features(im, sigma_max = 10)
    # predict
    predicted_segmentation = rs.predict_segmentor(clf, features)
    # clean detected roots
    roots = rs.clean_predicted_roots(predicted_segmentation, small_objects_threshold=150, closing_diameter = 4)
    # compute root properties
    results_df = rs.measure_roots(roots, root_thickness = 7, minimalBranchLength = 10)
    results_df["fname"] = fname
    # append to results list
    all_results.append(results_df)
    # save image for quality check
    rs.save_detected_roots_im(roots, im, fname[:-4] + "result.png", root_thickness = 7, minimalBranchLength = 10)


pd.concat(all_results).to_excel("../sample_results/measurements.xlsx")
